In [458]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from braindecode.models import EEGConformer
import pandas as pd
from collections import OrderedDict
from scipy.sparse import coo_matrix
import math
import importlib
import GIN_model
importlib.reload(GIN_model)
from GIN_model  import ShallowGINNet
from torchinfo import summary

In [459]:
in_chans = 22
n_classes = 4
input_window_samples = 1125
model = ShallowGINNet(in_chans,n_classes,input_window_samples)
#ref_model = ShallowFBCSPNet(in_chans,n_classes,input_window_samples)

In [460]:
print(model)

ShallowGINNet(
  (gin): GIN(22, 128, num_layers=2)
  (spatial): Conv2d(1, 1, kernel_size=(64, 1), stride=(1, 1))
  (pool): AvgPool1d(kernel_size=(100,), stride=(100,), padding=(0,))
  (batch_norm): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): LazyLinear(in_features=0, out_features=4, bias=True)
)


In [461]:
import torch
from torch_geometric.data import Data, Batch

# Number of nodes per graph
num_nodes = 1125  

# Construct a single graph's edge index
edge_index = torch.tensor([
    [i for i in range(num_nodes - 1)] + [i + 1 for i in range(num_nodes - 1)],
    [i + 1 for i in range(num_nodes - 1)] + [i for i in range(num_nodes - 1)]
], dtype=torch.long)

# Batch size
batch_size = 3  

# Repeat edge index for batching
edge_index = edge_index.repeat(1, batch_size)

# Create batch indices
batch = torch.tensor([i for i in range(batch_size) for _ in range(num_nodes)])

# Create node features (random example)
x = torch.rand((batch_size * num_nodes, 22))  # Assuming 22 node features per node

# Wrap into PyG data object
data_list = [Data(x=x[i*num_nodes:(i+1)*num_nodes], edge_index=edge_index) for i in range(batch_size)]
batched_data = Batch.from_data_list(data_list)

In [462]:
# Dummy input matching the expected input size
dummy_input = torch.rand(batch_size, 22, num_nodes)  # Ensure it matches your model's input size

# Ensure edge_index is properly formatted for PyG models
dummy_edge_index = torch.randint(0, num_nodes, (2, num_nodes * 2))  # Example edges

# Forward pass to initialize lazy layers
with torch.no_grad():
    model(dummy_input, edge_index=dummy_edge_index)  # Make sure your model takes edge_index


In [463]:
summary(model, input_size=(1, 22, 1125), col_names=["input_size", "output_size", "num_params", "kernel_size"],edge_index=edge_index)


Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Kernel Shape
ShallowGINNet                                 [1, 22, 1125]             [1, 4]                    --                        --
├─GIN: 1-1                                    [1, 1125, 22]             [1, 1125, 128]            --                        --
│    └─ModuleList: 2-5                        --                        --                        (recursive)               --
│    │    └─GINConv: 3-1                      [1, 1125, 22]             [1, 1125, 128]            19,456                    --
│    │    └─GINConv: 3-5                      --                        --                        (recursive)               --
│    │    └─GINConv: 3-3                      --                        --                        (recursive)               --
│    └─ModuleList: 2-2                        --                        --                        -- 